In [1]:
import psycopg2 as ps
from env_vars import *
import spotipy

import spotipy
import spotipy.util as util
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler, Normalizer
import pandas as pd
from pandas.io.json import json_normalize
from flask import jsonify
from joblib import load
import pickle
import numpy as np
from flask import request
from joblib import dump
from joblib import load
import pandas as pd

from env_vars import * 

In [2]:
import os
import sys
import json
import spotipy
import webbrowser
import spotipy.util as util
from json.decoder import JSONDecodeError
from env_vars import * 

# USERNAME = 'rileypence4' #your spotify username
# CLIENT_ID = '47c4f0d3dcaa433c9ef4ec6686e5b1f1' #set at your developer account
# CLIENT_SECRET = '65812a5fb52b4a3486f59ec672d499ff' #set at your developer account
# REDIRECT_URI = 'https://google.com/' #set at your developer account, usually "http://localhost:8000"
SCOPE = 'user-library-read' # or else
# # ps. REDIRECT_URI is crucial here. if http://localhost:8000 is not set, or with a single '/' misplaced, it will generate a connection error.

# then pass them:

token = util.prompt_for_user_token(username = USERNAME, 
                                   scope = SCOPE, 
                                   client_id = CLIENT_ID, 
                                   client_secret = CLIENT_SECRET, 
                                   redirect_uri = REDIRECT_URI)

if token:
   sp = spotipy.Spotify(auth=token)



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=b954b92b9a674e6a9cf7322649da82d1&response_type=code&redirect_uri=https%3A%2F%2Fwww.google.com%2F&scope=user-library-read in your browser




Enter the URL you were redirected to:  https://www.google.com/






            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=b954b92b9a674e6a9cf7322649da82d1&response_type=code&redirect_uri=https%3A%2F%2Fwww.google.com%2F&scope=user-library-read in your browser




Enter the URL you were redirected to:  https://www.google.com/


SpotifyOauthError: Bad Request

In [2]:
# def reconnect():
#     # create connection and cursor    
#     conn = ps.connect(host=POSTGRES_ADDRESS,
#                       database=POSTGRES_DBNAME,
#                       user=POSTGRES_USERNAME,
#                       password=POSTGRES_PASSWORD,
#                       port=POSTGRES_PORT)
#     cur = conn.cursor()
#     return cur

In [3]:
def reconnect():
    conn = ps.connect(host=POSTGRES_ADDRESS,
                  database=POSTGRES_DBNAME,
                  user=POSTGRES_USERNAME,
                  password=POSTGRES_PASSWORD,
                  port=POSTGRES_PORT)
    cur = conn.cursor()

In [4]:
class Sound_Drip:
    
    def __init__(self, token):
        self.token = token
    
    def instantiate_sp(self,token):
        sp = spotipy.Spotify(auth=token)
        return sp

    def get_user_song_id(self,sp):
        results = sp.current_user_saved_tracks()
        genre = []
        counter = 0 
        for song_number in range(0,19):
            counter += 1 
            song_id = results['items'][song_number]['track']['id']
            artist_id = sd.get_artist_id(song_id)
            genre = sd.get_genres(artist_id)
            if genre != []:
                break
        return song_id

    def get_acoustical_features(self,song_id,sp):
        acoustical_features = sp.audio_features(song_id)[0]
        return acoustical_features

    def get_popularity(self, song_id):
        popularity =  sp.track(song_id)['popularity']
        return popularity

    def get_artist_id(self, song_id):
        artist = sp.track(song_id)['artists'][0]['id']
        return artist

    def get_genres(self, artist):
        genre = sp.artist(artist)['genres']
        return genre
    
    def create_feature_object(self,popularity, acoustical_features):
        popularity_dict = {'popularity': popularity}
        song_features = acoustical_features
        song_features.update(popularity_dict)
        song_features = {
    "audio_features": {
        key: song_features[key] for key in song_features.keys() & {
            'popularity',
            'acousticness',
            'danceability',
            'energy',
            'instrumentalness',
            'key',
            'liveness',
            'loudness',
            'mode',
            'speechiness',
            'tempo',
            'time_signature',
            'valence'}}}

        df = pd.DataFrame.from_dict(json_normalize(song_features["audio_features"]),orient='columns')   
        df = df.reindex(sorted(df.columns), axis=1)
        return df
    
    def get_results(self,song_features_df):
        scaler = load("./models/scalar3.joblib")
        print('Scaling data...')
        data_scaled = scaler.transform(song_features_df)
        normalizer = Normalizer()
        data_normalized = normalizer.fit_transform(data_scaled)
        print('Loading pickled model...')
        model = load('./models/model5.joblib')
        results = model.kneighbors([data_normalized][0])[1:]
        print('results returned')
        return results[0]
    
    def filter_model(self,model_results,source_genre_list): 
        #loop takes KNN results and filters by source track genres
        print("filter for genres initiated")
        genre_array = pickle.load(open("./data/genres_array_2.pkl","rb"))
        filtered_list = []
        song_list_length = 20
        for output_song_index in model_results[0][1:]:
            output_genre_list = genre_array[output_song_index]
            for output_genre in output_genre_list:
                output_genre = output_genre.strip(" ")
                for source_genre in source_genre_list:
                    source_genre = "'" + source_genre + "'"
                    if source_genre == output_genre:
                        filtered_list.append(output_song_index)
                    else:
                        continue
        if len(set(filtered_list)) > song_list_length:
            print("filter found at least 20 genre matches")
            filtered_list = set(filtered_list)
            filtered_list = list(filtered_list)[0:20]
        else:
            counter = song_list_length - len(set(filtered_list))
            print(len(set(filtered_list)))
            print(counter)
            print(f'need to add {counter} items to final song output')
            for output_song_index in model_results[1:]:
                if output_song_index not in filtered_list:
                    if counter > 0:
                        filtered_list.append(output_song_index)
                        counter -= 1
                    else:
                        break
        print("filtered list with 20 unique song indices returned")
        return filtered_list
    
    def song_id_prediction_output(self,filtered_list): 
        similar_songs = []
        print('song_id_list loading...')
        song_id_array = pickle.load(open('./data/song_id_array3.pkl', 'rb'))
        print('song_id_list loaded')
        for song_row in filtered_list:
            song_id = song_id_array[song_row]
            similar_songs.append({'similarity': [.99], 'values': song_id})
        json_dict = {"songs": similar_songs}
        print("Results returned")
        return json_dict
    
    def get_user_ids(self, sp):
        current_user_dict = sp.current_user()
        display_name = current_user_dict['display_name']
        user_id = current_user_dict['id']
        return user_id, display_name

In [5]:
sd_obj = Sound_Drip('BQC1z9xdVqcDyY7e5JTdji-KWYUxiPQ8LnYDym1fLzP55dxRuzLzW3mX7cnCDmnWbIahdHnqePbLSSKX9Jz6FsrlAN8qRdbV3dIx-1VWUXkL4x3kGTHLjMfKcdjT1Kf7a0fGq-qpi9KeYF_0XENbUECWtfGfHfqvOmJN_Md9LidMXRitospOD1dDpii37EuD18nLQnxpkg')

In [6]:
sp = sd_obj.instantiate_sp(sd_obj.token)

In [7]:
sp

In [8]:
user_id, display_name = sd_obj.get_user_ids(sp)

SpotifyException: http status: 401, code:-1 - https://api.spotify.com/v1/me/:
 The access token expired

In [17]:
user_id, display_name

('basslaughter', 'Zhenya Warshavsky')

In [19]:
sd_obj.get_user_song_id(sp)

NameError: name 'sd' is not defined

In [7]:
sp = spotipy.Spotify('BQC1z9xdVqcDyY7e5JTdji-KWYUxiPQ8LnYDym1fLzP55dxRuzLzW3mX7cnCDmnWbIahdHnqePbLSSKX9Jz6FsrlAN8qRdbV3dIx-1VWUXkL4x3kGTHLjMfKcdjT1Kf7a0fGq-qpi9KeYF_0XENbUECWtfGfHfqvOmJN_Md9LidMXRitospOD1dDpii37EuD18nLQnxpkg')

In [19]:
print(sp.current_user()['display_name'])
sp.current_user()['id']

Zhenya Warshavsky


'basslaughter'

In [19]:
def get_user_ids(sp):
        current_user_dict = sp.current_user()
        display_name = current_user_dict['display_name']
        user_id = current_user_dict['id']
        return user_id, display_name

In [20]:
get_user_ids(sp)

('basslaughter', 'Zhenya Warshavsky')

In [42]:
# def run_query():
#     try:
#         cur.execute(query)
#     except Exception as e:
#         print (e.message)
#         return reconnect()

In [43]:
# run_query()

In [ ]:
# query = """SELECT * FROM users
#             WHERE schemaname != 'pg_catalog'
#             AND schemaname != 'information_schema';"""
# cur.execute(query)
# cur.fetchall()

In [4]:
query = """SELECT * FROM users;"""

cur.execute(query)
cur.fetchall()

[]

In [9]:
query = """SELECT * FROM recommendations"""

cur.execute(query)
cur.fetchall()

[]

In [ ]:
# cur.execute("""CREATE TABLE users
#                 (ID integer, 
#                 display_name varchar(50)
#                 );""")
# # Commit table creation
# conn.commit()

In [8]:
cur.execute("""CREATE TABLE recommendations
                (id serial PRIMARY KEY,
                userID varchar(50), 
                songID varchar(50),
                recDate DATE 
                )""")
# Commit table creation
conn.commit()

In [18]:
cur.execute("""CREATE TABLE account(
               user_id serial PRIMARY KEY,
               username VARCHAR (50) UNIQUE NOT NULL,
               password VARCHAR (50) NOT NULL,
               email VARCHAR (355) UNIQUE NOT NULL,
               created_on TIMESTAMP NOT NULL,
               last_login TIMESTAMP
               );""")
# Commit table creation
conn.commit()

In [ ]:
cursor.execute("""CREATE TABLE survival(
id SERIAL PRIMARY KEY,
survived integer)""")